1. Tirar as linhas que tem pelo menos 4 valores nulos ou maior-> ignorar essas instancias
2. Substituir tbd por -1 -> completar automaticamente
3. Isso é limpeza dos dados
4. Transformar cada classe em uma coluna
5. Separar jogos que são multi plataformas, correlação entre as avaliações

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

Uma vez que temos dois arquivos de csv, iremos juntar os dois, utilizando o concat.

In [2]:
dados1 = pd.read_csv( "trabalho2_dados_21.csv")
dados2 = pd.read_csv("trabalho3_dados_adicionais_21.csv")
df = pd.concat([dados1,dados2])

In [3]:
dados1.head()

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Green Lantern: Rise of the Manhunters,X360,Action,Warner Bros. Interactive Entertainment,0.10,7-Jun-11,59.0,17.0,7,22.0,Double Helix Games
1,Kinect Sports: Season Two,X360,Sports,Microsoft Game Studios,2.18,25-Oct-11,66.0,46.0,6.3,27.0,"BigPark, Rare Ltd."
2,Brothers in Arms: Double Time,Wii,Shooter,Ubisoft,0.12,23-Sep-08,45.0,13.0,6.6,17.0,Gearbox Software
3,Steambot Chronicles: Battle Tournament,PSP,Fighting,Irem Software Engineering,0.06,30-Jun-09,54.0,14.0,6.1,7.0,Irem
4,Tiger Woods PGA Tour 07,Wii,Sports,Electronic Arts,0.46,13-Mar-07,71.0,41.0,6.9,44.0,EA Redwood Shores


In [4]:
dados2.head()

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Tiger Woods PGA Tour 06,X360,Sports,Electronic Arts,0.30,16-Nov-05,71.0,36.0,7.4,8.0,EA Sports
1,Peter Jackson's King Kong: The Official Game o...,X360,Action,Ubisoft,0.23,21-Nov-05,80.0,57.0,7.2,60.0,Ubisoft Montpellier
2,Pinball Hall of Fame: The Gottlieb Collection,PSP,Misc,Play It,0.22,6-Dec-05,73.0,27.0,9.5,11.0,Farsight Studios
3,Tiger Woods PGA Tour 06,PSP,Sports,Electronic Arts,0.20,26-Sep-05,80.0,15.0,9,7.0,Team Fusion
4,Peter Jackson's King Kong: The Official Game o...,PSP,Action,Ubisoft,0.18,20-Dec-05,56.0,10.0,6.2,20.0,Phoenix Games Studio


Vamos analisar os tipo de cada variável no dataframe.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3196 entries, 0 to 1563
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   nome                3196 non-null   object 
 1   plataforma          3196 non-null   object 
 2   genero              3196 non-null   object 
 3   editora             3192 non-null   object 
 4   vendas              3196 non-null   float64
 5   lancamento          2490 non-null   object 
 6   avaliacao-criticos  2050 non-null   float64
 7   numero-criticos     2050 non-null   float64
 8   avaliacao-usuarios  2446 non-null   object 
 9   numero-usuarios     1944 non-null   float64
 10  fabricante          2473 non-null   object 
dtypes: float64(4), object(7)
memory usage: 299.6+ KB


Observer que a quantidade de valores em avaliacao-usuarios é maior que o numero de usuarios. Além disso, não possuímos as mesmas qauntidades em todas as variáveis. Dependendo do modelo a ser escolhido, essa diferença de tamanho pode representar uma dificuldade.

In [6]:
df.head()

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Green Lantern: Rise of the Manhunters,X360,Action,Warner Bros. Interactive Entertainment,0.10,7-Jun-11,59.0,17.0,7,22.0,Double Helix Games
1,Kinect Sports: Season Two,X360,Sports,Microsoft Game Studios,2.18,25-Oct-11,66.0,46.0,6.3,27.0,"BigPark, Rare Ltd."
2,Brothers in Arms: Double Time,Wii,Shooter,Ubisoft,0.12,23-Sep-08,45.0,13.0,6.6,17.0,Gearbox Software
3,Steambot Chronicles: Battle Tournament,PSP,Fighting,Irem Software Engineering,0.06,30-Jun-09,54.0,14.0,6.1,7.0,Irem
4,Tiger Woods PGA Tour 07,Wii,Sports,Electronic Arts,0.46,13-Mar-07,71.0,41.0,6.9,44.0,EA Redwood Shores


Há algumas variáveis que seria interessante transformá-las, para que possam ser utilizadas em um modelo. São elas: plataforma, pois nós possuímos 9 classes e seria interessante termos a informação dos tipos de plataformas que um jogo foi produzido em uma única instância; genero seria interessante transformar em numérico; avaliacao-usuarios deve ser um valor numérico;

In [7]:
df['plataforma'].unique()

array(['X360', 'Wii', 'PSP', 'PS3', 'PS4', '3DS', 'PSV', 'XOne', 'WiiU'],
      dtype=object)

Antes de passarmos para essas etapas citadas, iremos limpar os dados.

Como vamos começar a retirar os valores nulos, irei passar a coluna de avaliação dos usuários para float e substituir 'tbd' por -1.

In [8]:
df['avaliacao-usuarios'] = pd.to_numeric(df['avaliacao-usuarios'].replace('tbd','-1'),errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3196 entries, 0 to 1563
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   nome                3196 non-null   object 
 1   plataforma          3196 non-null   object 
 2   genero              3196 non-null   object 
 3   editora             3192 non-null   object 
 4   vendas              3196 non-null   float64
 5   lancamento          2490 non-null   object 
 6   avaliacao-criticos  2050 non-null   float64
 7   numero-criticos     2050 non-null   float64
 8   avaliacao-usuarios  2446 non-null   float64
 9   numero-usuarios     1944 non-null   float64
 10  fabricante          2473 non-null   object 
dtypes: float64(5), object(6)
memory usage: 299.6+ KB


Ao olhar os arquivos csv, pudemos observar que as instâncias que possuem mais de 3 campos vazios, colunas, podem ser retirados do dataframe, pois estas instâncias não possuem valores de avaliação. 

In [9]:
display(df)

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Green Lantern: Rise of the Manhunters,X360,Action,Warner Bros. Interactive Entertainment,0.10,7-Jun-11,59.0,17.0,7.0,22.0,Double Helix Games
1,Kinect Sports: Season Two,X360,Sports,Microsoft Game Studios,2.18,25-Oct-11,66.0,46.0,6.3,27.0,"BigPark, Rare Ltd."
2,Brothers in Arms: Double Time,Wii,Shooter,Ubisoft,0.12,23-Sep-08,45.0,13.0,6.6,17.0,Gearbox Software
3,Steambot Chronicles: Battle Tournament,PSP,Fighting,Irem Software Engineering,0.06,30-Jun-09,54.0,14.0,6.1,7.0,Irem
4,Tiger Woods PGA Tour 07,Wii,Sports,Electronic Arts,0.46,13-Mar-07,71.0,41.0,6.9,44.0,EA Redwood Shores
...,...,...,...,...,...,...,...,...,...,...,...
1559,Port Royale 3,X360,Simulation,Kalypso Media,0.04,9-Oct-12,56.0,11.0,4.5,6.0,Kalypso
1560,Major League Baseball 2K8,PSP,Sports,Unknown,0.03,3-Mar-08,63.0,5.0,-1.0,NaN,Kush Games
1561,Monster Hunter Frontier Online,PS3,Role-Playing,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN
1562,Battle vs. Chess,PS3,Misc,TopWare Interactive,0.03,TBA,NaN,NaN,NaN,NaN,TopWare Interactive


In [15]:
## Retirando as instâncias que possuem mais de 3 campos sem preencher
indice = lambda x: df.iloc[x].isnull().sum()
guarda_index=[]
for i in df.index:
    if indice(i)>=3:
        guarda_index.append(i)
df.drop(guarda_index, inplace=True)
#renomear os indices
indice_novo = [i for i in range(len(df))]
for i in range(len(df)):
    df.rename({df.index[i]:indice_novo[i]}, inplace=True,axis='index')

In [16]:
df.iloc[0]

nome                   Green Lantern: Rise of the Manhunters
plataforma                                              X360
genero                                                Action
editora               Warner Bros. Interactive Entertainment
vendas                                                   0.1
lancamento                                          7-Jun-11
avaliacao-criticos                                        59
numero-criticos                                           17
avaliacao-usuarios                                         7
numero-usuarios                                           22
fabricante                                Double Helix Games
Name: 748, dtype: object

In [17]:
df

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
748,Green Lantern: Rise of the Manhunters,X360,Action,Warner Bros. Interactive Entertainment,0.10,7-Jun-11,59.0,17.0,7.0,22.0,Double Helix Games
749,Kinect Sports: Season Two,X360,Sports,Microsoft Game Studios,2.18,25-Oct-11,66.0,46.0,6.3,27.0,"BigPark, Rare Ltd."
750,Brothers in Arms: Double Time,Wii,Shooter,Ubisoft,0.12,23-Sep-08,45.0,13.0,6.6,17.0,Gearbox Software
751,Steambot Chronicles: Battle Tournament,PSP,Fighting,Irem Software Engineering,0.06,30-Jun-09,54.0,14.0,6.1,7.0,Irem
752,Tiger Woods PGA Tour 07,Wii,Sports,Electronic Arts,0.46,13-Mar-07,71.0,41.0,6.9,44.0,EA Redwood Shores
...,...,...,...,...,...,...,...,...,...,...,...
1450,Bejeweled 3,X360,Puzzle,Unknown,0.12,19-Oct-11,78.0,13.0,8.4,7.0,PopCap
1451,Get Fit with Mel B,X360,Sports,Black Bean Games,0.07,Q2 2011,57.0,7.0,NaN,NaN,Lightning Fish Games
1452,Captain America: Super Soldier,Wii,Action,Sega,0.05,19-Jul-11,57.0,16.0,6.4,5.0,High Voltage Software
1453,Port Royale 3,PS3,Simulation,Kalypso Media,0.04,9-Oct-12,NaN,NaN,8.5,4.0,Kalypso
